# Tutorial - Preparing measurement table

This tutorial takes you through the entire workflow of the [Biology][biology] module.

In [ ]:
import eds_scikit
import pandas as pd

## 1 - Load data <a id="load-data"></a>

!!! tip "Big volume"
    Measurement table can be large. Do not forget to set proper spark config.

In [ ]:
to_add_conf = [
    ("master", "yarn"),
    ("deploy_mode", "client"),
    ("spark.driver.memory", ...),
    ("spark.executor.memory", ...),
    ("spark.executor.cores", ...),
    ("spark.executor.memoryOverhead", ...),
    ("spark.driver.maxResultSize", ...)
    ...
]

spark, sc, sql = eds_scikit.improve_performances(to_add_conf=to_add_conf)

from eds_scikit.io.hive import HiveData

In [ ]:
data = HiveData(
    spark_session=spark,
    database_name="cse_xxxxxxx_xxxxxxx",
    tables_to_load=[
        "care_site",
        "concept",
        "visit_occurrence",
        "measurement",
        "concept_relationship",
    ],
)

## 2 - Quick use : Preparing measurement table <a id="quick-use"></a>

### a) Define biology concept-sets <a id="define-biology-concept-set"></a>

In order to work on the measurements of interest, you can extract a list of concepts-sets by:

- Selecting [default concepts-sets](../../datasets/concepts-sets.md) provided in the library.
- Modifying the codes of a selected default concepts-set.
- Creating a concepts-set from scratch.

__Code selection can be tricky. See <a href="#concept-codes-explorer">Concept codes relationships exploration</a> section for more details on how to select them.__

In [ ]:
from eds_scikit.biology import ConceptsSet

# Creating Concept-Set
custom_leukocytes = ConceptsSet("Custom_Leukocytes")

custom_leukocytes.add_concept_codes(
    concept_codes=['A0174', 'H6740', 'C8824'], 
    terminology='GLIMS_ANABIO' 
)
custom_leukocytes.add_concept_codes(
    concept_codes=['6690-2'], 
    terminology='ITM_LOINC'
)

# Importing Concept-Set (see. 4.b for details on existing concepts sets)
glucose_blood = ConceptsSet("Glucose_Blood")

In [ ]:
concepts_sets = [
    custom_leukocytes, 
    glucose_blood
]

### b) Prepare measurements <a id="prepare-measurements"></a>

!!! tip "Lazy execution"
    Execution will be lazy, except if ```convert_units=True```.

In [ ]:
from eds_scikit.biology import prepare_measurement_table

In [ ]:
measurement_bioclean = prepare_measurement_table(data,
                                                 start_date="2022-01-01", end_date="2022-05-01",
                                                 concept_sets=concepts_sets,
                                                 convert_units=False,
                                                 get_all_terminologies=True
                                                )

__Now you have your measurement table mapped with concept set terminology.__ Next sections are about measurement codes analysis, units and plots.

## 3 - Detailed use : Analysing measurement table<a id="detailed-use"></a>

### a) Measurements statistics table <a id="stat-table"></a>

In [ ]:
from eds_scikit.biology import measurement_values_summary

In [ ]:
stats_summary = measurement_values_summary(measurement_bioclean, 
                                           category_cols=["concept_set", "GLIMS_ANABIO_concept_code", "GLIMS_LOINC_concept_code"], 
                                           value_column="value_as_number", 
                                           unit_column="unit_source_value")

stats_summary

|                   |       |     |         |   range_low_anomaly_count |   range_high_anomaly_count |   measurement_count |   value_as_number_count |   value_as_number_mean |   value_as_number_std |   value_as_number_min |   value_as_number_25% |   value_as_number_50% |   value_as_number_75% |   value_as_number_max |
|:------------------|:------|----:|:--------|--------------------------:|---------------------------:|--------------------:|------------------------:|-----------------------:|----------------------:|----------------------:|----------------------:|----------------------:|----------------------:|----------------------:|
| Custom_Leukocytes | A0174 | 148 | x10*9/l |                       813 |                       1099 |               11857 |                   11857 |                     21 |                    18 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Custom_Leukocytes | C8824 | 121 | x10*9/l |                      1166 |                       1196 |               11821 |                   11821 |                     20 |                    20 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Custom_Leukocytes | C9784 |  83 | x10*9/l |                       935 |                        902 |               11082 |                   11082 |                     10 |                    16 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Glucose_Blood     | A0141 | 147 | mmol/l  |                      1179 |                        976 |               12811 |                   12811 |                     27 |                    20 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Glucose_Blood     | A7338 |  43 | mmol/l  |                       819 |                        755 |               11312 |                   11312 |                     27 |                    13 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Glucose_Blood     | A8424 | 176 | mmol/l  |                       916 |                        936 |               12020 |                   12020 |                     14 |                    25 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Glucose_Blood     | B9553 | 107 | mmol/l  |                       794 |                       1046 |               13409 |                   13409 |                     17 |                    19 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Glucose_Blood     | C0565 |  50 | g/l     |                       586 |                       1030 |                9720 |                    9720 |                     18 |                    26 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Glucose_Blood     | C7236 |  64 | g/l     |                      1121 |                        882 |               13685 |                   13685 |                     25 |                    12 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Glucose_Blood     | E7312 |  51 | mg/dl   |                      1266 |                        874 |               10971 |                   10971 |                     15 |                    17 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Glucose_Blood     | E9889 | 131 | mg/dl   |                       992 |                        833 |                6685 |                    6685 |                     15 |                    24 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Glucose_Blood     | F2622 | 182 | mmol/l  |                       865 |                        698 |               10596 |                   10596 |                     16 |                     7 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Glucose_Blood     | H7323 |  92 | mmol/l  |                      1286 |                        860 |                9898 |                    9898 |                     22 |                    26 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Glucose_Blood     | H7324 |  88 | mmol/l  |                       976 |                       1040 |               10452 |                   10452 |                     22 |                    18 |                     0 |                    25 |                    50 |                    75 |                   100 |
| Glucose_Blood     | J7401 | 132 | mmol/l  |                       850 |                       1206 |               12363 |                   12363 |                     18 |                     0 |                     0 |                    25 |                    50 |                    75 |                   100 |

### b) Measurements units correction <a id="units-correction"></a>

In [ ]:
glucose_blood.add_conversion("mol", "g", 180)
glucose_blood.add_target_unit("mmol/l")

concepts_sets = [glucose_blood, custom_leukocytes]

In [ ]:
measurement_bioclean = prepare_measurement_table(data, 
                                                 start_date="2022-01-01", end_date="2022-05-01",
                                                 concept_sets=concepts_sets,
                                                 convert_units=True, 
                                                 get_all_terminologies=False 
                                                )

In [ ]:
stats_summary = measurement_values_summary(measurement_bioclean, 
                                           category_cols=["concept_set", "GLIMS_ANABIO_concept_code"], 
                                           value_column="value_as_number_normalized", #converted
                                           unit_column="unit_source_value_normalized")

stats_summary

|                   |       |     |        |   range_low_anomaly_count |   range_high_anomaly_count |   measurement_count |   value_as_number_normalized_count |   value_as_number_normalized_mean |   value_as_number_normalized_std |   value_as_number_normalized_min |   value_as_number_normalized_25% |   value_as_number_normalized_50% |   value_as_number_normalized_75% |   value_as_number_normalized_max |   value_as_number_count |   value_as_number_mean |   value_as_number_std |   value_as_number_min |   value_as_number_max |   value_as_number_25% |   value_as_number_50% |   value_as_number_75% |
|:------------------|:------|----:|:-------|--------------------------:|---------------------------:|--------------------:|-----------------------------------:|----------------------------------:|---------------------------------:|---------------------------------:|---------------------------------:|---------------------------------:|---------------------------------:|---------------------------------:|------------------------:|-----------------------:|----------------------:|----------------------:|----------------------:|----------------------:|----------------------:|----------------------:|
| Custom_Leukocytes | A0174 | 215 | 10*6/l |                      1259 |                       1131 |               16187 |                        1.37744e+06 |                        8961.12    |                       9214.09    |                         0        |                       5600       |                       7690       |                      10590       |                      969720      |                   16187 |                     27 |                    17 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Custom_Leukocytes | C8824 |  41 | 10*6/l |                      1092 |                       1286 |               12351 |                       78           |                       27487.3     |                      85884.3     |                      1100        |                       5000       |                       9700       |                      20200       |                      697500      |                   12351 |                     18 |                    16 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Custom_Leukocytes | C9784 |  80 | 10*6/l |                       886 |                        855 |               13957 |                    96962           |                        9089.04    |                      12083.2     |                         0        |                       5600       |                       7600       |                      10460       |                      725400      |                   13957 |                     21 |                     4 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Glucose_Blood     | A0141 | 103 | mmol/l |                       808 |                        821 |               13563 |                   105958           |                           6.54161 |                          4.85914 |                         0.1      |                          4.8     |                          5.6     |                          7       |                        1007      |                   13563 |                     29 |                     8 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Glucose_Blood     | A7338 | 197 | mmol/l |                       895 |                       1064 |               12503 |                    35302           |                           5.83727 |                          2.23913 |                         0.3      |                          4.72    |                          5.2     |                          6.03    |                          49.68   |                   12503 |                     20 |                     8 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Glucose_Blood     | A8424 |  88 | mmol/l |                       801 |                        959 |               10270 |                    18377           |                           6.25654 |                          3.35737 |                         0.1      |                          4.7     |                          5.4     |                          6.8     |                          65.5    |                   10270 |                     17 |                    24 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Glucose_Blood     | B9553 | 200 | mmol/l |                       999 |                        950 |               16199 |                     3940           |                           6.55372 |                          3.23465 |                         1.8      |                          4.8     |                          5.5     |                          7.2     |                          44.8    |                   16199 |                     19 |                    22 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Glucose_Blood     | C0565 | 193 | mmol/l |                       991 |                       1228 |               10263 |                    13431           |                           6.02093 |                          2.49668 |                         0.305556 |                          4.77778 |                          5.27778 |                          6.22222 |                          50.1833 |                   10263 |                     30 |                    22 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Glucose_Blood     | C7236 |  31 | mmol/l |                      1090 |                        970 |               13942 |                     5246           |                           7.03372 |                          3.79997 |                         0.455556 |                          5       |                          5.77778 |                          7.61111 |                          58.1611 |                   13942 |                     15 |                    21 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Glucose_Blood     | E7312 | 182 | mmol/l |                       678 |                       1334 |               16282 |                    16757           |                           6.44004 |                          3.48772 |                         0.6      |                          4.7     |                          5.40556 |                          6.80556 |                          53.4278 |                   16282 |                     19 |                    22 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Glucose_Blood     | E9889 |  56 | mmol/l |                      1201 |                        801 |               14401 |                     3297           |                           6.01896 |                          2.47918 |                         1.3      |                          4.7     |                          5.3     |                          6.3     |                          36.2    |                   14401 |                     19 |                    14 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Glucose_Blood     | F2622 | 135 | mmol/l |                      1049 |                        905 |               14913 |                       15           |                           3.19933 |                          1.40811 |                         0.23     |                          2.01    |                          3.56    |                          4.41    |                           4.9    |                   14913 |                     26 |                    15 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Glucose_Blood     | H7323 |  69 | mmol/l |                       827 |                        967 |               12758 |                     3262           |                           7.48074 |                          4.01494 |                         1.3      |                          5.2     |                          6.2     |                          8.4     |                          73.4    |                   12758 |                     11 |                    25 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Glucose_Blood     | H7324 |  97 | mmol/l |                       861 |                        498 |               10581 |                     2479           |                           6.259   |                          2.77276 |                         1.5      |                          4.9     |                          5.4     |                          6.4     |                          39.4    |                   10581 |                     16 |                    25 |                     0 |                   100 |                    25 |                    50 |                    75 |
| Glucose_Blood     | J7401 | 126 | mmol/l |                      1236 |                        936 |               16228 |                    10964           |                           6.91422 |                          3.3815  |                         0.1      |                          5       |                          5.8     |                          7.7     |                          59.5    |                   16228 |                     22 |                    23 |                     0 |                   100 |                    25 |                    50 |                    75 |


### c) Plot biology summary <a id="plot-summary"></a>

Applying ```plot_biology_summary``` to computed measurement dataframe, merged with care sites, allows to generate nice exploration plots such as :

- [Interactive volumetry](../../_static/biology/viz/interactive_volumetry.html)

- [Interactive distribution](../../_static/biology/viz/interactive_distribution.html)

In [ ]:
from eds_scikit.biology import plot_biology_summary

In [ ]:
measurement_bioclean = measurement_bioclean.merge(data.visit_occurrence[["care_site_id", "visit_occurrence_id"]], on="visit_occurrence_id")
measurement_bioclean = measurement_bioclean.merge(data.care_site[["care_site_id", "care_site_short_name"]], on="care_site_id")

In [ ]:
plot_biology_summary(measurement_bioclean, value_column="value_as_number_normalized") 

## 4 - Further : Concept Codes, Concepts Sets and Units <a id="further"></a>

### 1 - Concept codes relationships exploration <a id="concept-codes-explorer"></a>

Concept codes relationships can be tricky to understand and to manipulate. Function ```prepare_biology_relationship_table``` allows to build __mapping dataframe between main AP-HP biology referential__.

See ```io.settings.measurement_config["mapping"]``` and ```io.settings.measurement_config["source_terminologies"]``` configurations for mapping details.

In [ ]:
from eds_scikit.biology import prepare_biology_relationship_table

biology_relationship_table = prepare_biology_relationship_table(data)
biology_relationship_table = biology_relationship_table.to_pandas()

Relationship between codes from different referentials.

In [ ]:
columns = [col for col in biology_relationship_table.columns if "concept_code" in col]

biology_relationship_table[biology_relationship_table.GLIMS_ANABIO_concept_code.isin(['A0174', 'H6740', 'C8824'])][columns].drop_duplicates()

|   ANALYSES_LABORATOIRE_concept_code | GLIMS_ANABIO_concept_code   | GLIMS_LOINC_concept_code   | ITM_ANABIO_concept_code   | ITM_LOINC_concept_code   |
|------------------------------------:|:----------------------------|:---------------------------|:--------------------------|:-------------------------|
|                                   0 | C8824                       | 33256-9                    | Unknown                   | Unknown                  |
|                                   1 | A0174                       | 6690-2                     | A0174                     | 6690-2                   |
|                                   1 | A0174                       | 26464-8                    | A0174                     | 6690-2                   |


In [ ]:
biology_relationship_table[biology_relationship_table.GLIMS_LOINC_concept_code.isin(['33256-9', '6690-2', '26464-8'])][columns].drop_duplicates()

|   ANALYSES_LABORATOIRE_concept_code | GLIMS_ANABIO_concept_code   | GLIMS_LOINC_concept_code   | ITM_ANABIO_concept_code   | ITM_LOINC_concept_code   |
|------------------------------------:|:----------------------------|:---------------------------|:--------------------------|:-------------------------|
|                                   4 | E4358                       | 6690-2                     | Unknown                   | Unknown                  |
|                                   2 | C9097                       | 26464-8                    | Unknown                   | Unknown                  |
|                                   6 | K3232                       | 6690-2                     | Unknown                   | Unknown                  |
|                                   5 | E6953                       | 26464-8                    | Unknown                   | Unknown                  |
|                                   1 | C8824                       | 33256-9                    | Unknown                   | Unknown                  |
|                                   4 | E4358                       | 26464-8                    | Unknown                   | Unknown                  |
|                                   5 | E6953                       | 6690-2                     | Unknown                   | Unknown                  |
|                                   7 | K6094                       | 6690-2                     | Unknown                   | Unknown                  |
|                                   0 | C9784                       | 6690-2                     | C9784                     | 6690-2                   |
|                                   0 | C9784                       | 26464-8                    | C9784                     | 6690-2                   |
|                                   3 | A0174                       | 6690-2                     | A0174                     | 6690-2                   |
|                                   3 | A0174                       | 26464-8                    | A0174                     | 6690-2                   |


### 2 - Concepts-Sets <a id="concepts-sets"></a>

To get all availables concepts sets see `datasets.default_concepts_sets`. More details about their definition and how they are build can be found in this [section](#concepts-sets).


In [22]:
from eds_scikit import datasets
from eds_scikit.biology import ConceptsSet

In [23]:
print(ConceptsSet("Troponin").concept_codes)

{'GLIMS_ANABIO': ['A0283', 'C5560', 'F9934', 'E6954', 'L3534', 'G7716', 'J5184', 'A3832', 'E7249']}


### 3 - Units <a id="units"></a>

Units module makes conversion between units easier. It uses configuration files `datasets.units` and `datasets.elements`.

In [ ]:
from eds_scikit import datasets

In [24]:
from eds_scikit.biology import Units

In [25]:
units = Units()

print("L to ml : ", units.convert_unit("L", "ml"))
print("m/s to m/h : ", units.convert_unit("m/s", "m/h"))
print("g to mol : ", units.convert_unit("g", "mol"))
units.add_conversion("mol", "g", 180)
print("g to mol : ", units.convert_unit("g", "mol"))

L to ml :  1000.0
m/s to m/h :  3600.000000001008
g to mol :  nan
g to mol :  0.005555555555555556
